In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from vgg16 import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def _image_parser (path, label):
    
    # Read Image
    image_file = tf.read_file(path)
    image_decoded = tf.image.decode_jpeg(image_file, channels=3)
    image_resized = tf.image.resize_images(image_decoded, INPUT_IMAGE_SIZE)

    return image_resized, label

In [3]:
# Misc
LOG_DIR = './log/'
MODEL_ID = 0.001
IMGNET_PRE_MODEL = './imagenet_pretrained_20/model.ckpt'

# Training Parameters
SHUFFLE_BUFFER= 250
LEARNING_RATE = 0.001
MOMENTUM = 0.9
NUM_EPOCHS = 1
NUM_STEPS = 5000
VAL_STEPS = 100
BATCH_SIZE = 256
DISPLAY_STEP = 1
INPUT_IMAGE_SIZE = [256, 256]
NUM_CLASSES = 20
KEEP_RATE = 0.75

In [4]:
# Data preparation
test = pd.read_csv('../test20.txt', delimiter=' ', header=None).sample(frac=1)

In [5]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    keep_prob = tf.placeholder(tf.float32)
    rand_crop = tf.placeholder(tf.bool)
    
    # Dataset
    test_ds = tf.data.Dataset.from_tensor_slices(
        (np.array(test[0]), np.array(test[1]))).map(_image_parser).batch(BATCH_SIZE)
    
    # Dataset Iterator
    iterator = test_ds.make_one_shot_iterator()
    x, y = iterator.get_next()
    
    # Image Summary
    # tf.summary.image('input', x, 5)
    
    # Build Model
    vgg = MY_VGG16(x=x, keep_rate=keep_prob, num_classes=NUM_CLASSES, 
                   batch_size=BATCH_SIZE, 
                   skip_layers=['fc7', 'fc8'], weights_path='../vgg16.npy',
                   retrain=True, random_crop=rand_crop)
    vgg.build()
    
    # Image Summary
    # tf.summary.image('input', vgg.final_input, 5)
    
    # Logits and Predictions
    logits = vgg.logits
    
    with tf.variable_scope('predictions'):
        pred_classes  = tf.argmax(logits, axis=1, name='prediction_label')
        softmax = tf.nn.softmax(logits, name='softmax')
        tf.summary.histogram('prediction_label', pred_classes)
        tf.summary.histogram('softmax', softmax)

    # Loss
    with tf.variable_scope('cross_entropy_loss'):   
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='sparse_cross_entropy')
        loss_op = tf.reduce_mean(cross_entropy, name='mean_loss')
        tf.summary.scalar('cross_entropy_loss', loss_op)
    
    # Evaluation
    with tf.variable_scope('accuracy'):
#         true_labels = tf.argmax(y, 1, name='true_label')
        correct_pred = tf.equal(pred_classes, y, name='true_pred_equal')
        accuracy_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='mean_accuracy')
        tf.summary.scalar('accuracy', accuracy_op)
    
    # Global Initializer
    global_init = tf.global_variables_initializer()
    
    # Merge Summary
    summary = tf.summary.merge_all()
    
    # Global saver
    saver = tf.train.Saver()

In [7]:
with tf.Session(graph=model_graph) as sess:
    
    # Debugger
#     sess = tf_debug.TensorBoardDebugWrapperSession(sess, "marr:7000")
    
    # Restore model
    print('Loading model')
    saver.restore(sess, './log/model_0.001/epoch.model.ckpt-5')
    #saver.restore(sess, './imagenet_pretrained_20/model.ckpt')
    print('Model restored.')
    
    print('Start testing.')

    # Testing
    test_acc = 0
    test_loss = 0
    testing_iter = 0

    while True:        
        try:
            loss_val, acc = sess.run([loss_op, accuracy_op], feed_dict={keep_prob: 1.0, rand_crop: False})
            test_acc += acc
            test_loss += loss_val
            testing_iter += 1
            
            print("Testing step" + str(testing_iter) + ", Minibatch Loss= " + \
                "{:.4f}".format(loss_val) + ", Minibatch Accuracy= " + \
                "{:.3f}".format(acc))
            
        except tf.errors.OutOfRangeError:
            break

    assert testing_iter != 0
    print("Overall Testing" + ", Loss= " + \
          "{:.4f}".format(test_loss/testing_iter) + ", Accuracy= " + \
          "{:.3f}".format(test_acc/testing_iter))
    
    print('Done.')

Loading model
INFO:tensorflow:Restoring parameters from ./log/model_0.001/epoch.model.ckpt-5
Model restored.
Start testing.
Testing step1, Minibatch Loss= 0.7241, Minibatch Accuracy= 0.836
Testing step2, Minibatch Loss= 0.4762, Minibatch Accuracy= 0.863
Testing step3, Minibatch Loss= 0.7536, Minibatch Accuracy= 0.855
Testing step4, Minibatch Loss= 0.4858, Minibatch Accuracy= 0.863
Testing step5, Minibatch Loss= 0.5781, Minibatch Accuracy= 0.867
Testing step6, Minibatch Loss= 0.5286, Minibatch Accuracy= 0.867
Testing step7, Minibatch Loss= 0.8923, Minibatch Accuracy= 0.805
Testing step8, Minibatch Loss= 0.5104, Minibatch Accuracy= 0.863
Testing step9, Minibatch Loss= 0.5579, Minibatch Accuracy= 0.855
Testing step10, Minibatch Loss= 0.8366, Minibatch Accuracy= 0.820
Testing step11, Minibatch Loss= 0.7325, Minibatch Accuracy= 0.828
Testing step12, Minibatch Loss= 0.6737, Minibatch Accuracy= 0.848
Testing step13, Minibatch Loss= 0.5944, Minibatch Accuracy= 0.863
Testing step14, Minibatch L